# Top Restaurantes do bairro Pinheiros/SP via Zomato

### Importação de pacotes:

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [67]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
response = requests.get("https://www.zomato.com/pt/sao-paulo-sp/best-restaurants", headers=headers)
html = response.content

### Entendendo o site:

In [68]:
type(html)

bytes

In [69]:
html = html.decode('utf-8')

In [70]:
type(html)

str

In [71]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

In [72]:
html = trata_html(html)

In [73]:
soup = BeautifulSoup(html, 'html.parser')

In [74]:
type(soup)

bs4.BeautifulSoup

In [75]:
# print(soup.prettify())

In [76]:
restaurantes = soup.find('div', {'class': 'ui cards'})

In [77]:
dados_restaurante = restaurantes.find('div', {'class':'col-s-16 col-m-12 pl0'})
dados_restaurante

<div class="col-s-16 col-m-12 pl0"><div class="row"><div class="col-s-12"><div class="res-snippet-small-establishment mt5" style="margin-bottom: 7px;"><a class="zdark ttupper fontsize6" href="https://www.zomato.com/pt/sao-paulo-sp/refeições-de-luxo " title="Refeições de luxo em São Paulo">Refeições de luxo</a></div><a class="result-title hover_feedback zred bold ln24 fontsize0" data-result-type="ResCard_Name" href="https://www.zomato.com/pt/sao-paulo-sp/a-figueira-rubaiyat-jardim-paulista" title="figueira rubaiyat, Jardim Paulista">A Figueira Rubaiyat </a><div class="single-rating flex"><div class="flex"><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#1C1C1C;"></div><div class="star" data-icon="" style="color:#cfcfcf;"></div></div><span class="rating-value">4.4</span><span class="review-count medium">(51 Op

In [78]:
tipo_restaurante = restaurantes.find('div', {'class':'search-page-text clearfix row'})
tipo_restaurante

<div class="search-page-text clearfix row"><div class="clearfix"><span class="col-s-5 col-m-4 ttupper fontsize5 grey-text">Cozinhas: </span><span class="col-s-11 col-m-12 nowrap pl0"><a href="https://www.zomato.com/pt/sao-paulo-sp/restaurants/bbq" title="Churrasco">Churrasco</a>, <a href="https://www.zomato.com/pt/sao-paulo-sp/restaurants/grill" title="Grelhados">Grelhados</a>, <a href="https://www.zomato.com/pt/sao-paulo-sp/restaurants/brazilian" title="Brasileira">Brasileira</a></span></div><div class="res-cost clearfix"><span class="col-s-5 col-m-4 ttupper fontsize5 grey-text">Custo para dois:</span><span class="col-s-11 col-m-12 pl0">R$ 300</span></div><div class="res-timing clearfix" title="12:30 to 24:00 (Mon-Thu),12:30 to 01:00 (Fri-Sat),Closed (Sun)"><span class="col-s-5 col-m-4 ttupper fontsize5 grey-text left">Horário:</span><div class="col-s-11 col-m-12 pl0 search-grid-right-text"> 12:30 a 24:00 (Seg-Qui),12:30 a 01:00... </div><div class="clear"></div></div><div class="res-

In [79]:
card = []
card = {}

In [80]:
card['restaurante'] = [dados_restaurante.find('a', {'class':'result-title hover_feedback zred bold ln24 fontsize0'}).getText()]
card['nota'] = [dados_restaurante.find('span', {'class':'rating-value'}).getText()]
card['numero_de_avaliacoes'] = [dados_restaurante.find('span', {'class':'review-count medium'}).getText()]
card['tipo'] = [dados_restaurante.find('a', {'class':'zdark ttupper fontsize6'}).getText()]
card['bairro'] = [dados_restaurante.find('a', {'class':'ln24 search-page-text mr10 zblack search_result_subzone left'}).getText()]
card['cozinha'] = [tipo_restaurante.find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}).getText()]
card['custo_para_dois'] = [tipo_restaurante.find('span', {'class':'col-s-11 col-m-12 pl0'}).getText()]

In [81]:
card

{'restaurante': ['A Figueira Rubaiyat '],
 'nota': ['4.4'],
 'numero_de_avaliacoes': ['(51 Opiniões)'],
 'tipo': ['Refeições de luxo'],
 'bairro': ['Jardim Paulista'],
 'cozinha': ['Churrasco, Grelhados, Brasileira'],
 'custo_para_dois': ['R$ 300']}

In [82]:
restaurantes_pinheiros = pd.DataFrame(card, index=None)
restaurantes_pinheiros

,restaurante,nota,numero_de_avaliacoes,tipo,bairro,cozinha,custo_para_dois
0,A Figueira Rubaiyat,4.4,(51 Opiniões),Refeições de luxo,Jardim Paulista,"Churrasco, Grelhados, Brasileira",R$ 300


### Coletando os dados e transformando em um DataFrame:

In [83]:
# Declarando variável cards
cards = []

local = ''
pagina_final = 150

for i in range(pagina_final):

    # Obtendo o HTML:
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
    response = requests.get('https://www.zomato.com/pt/sao-paulo-sp/best-' + str(local) +'restaurants?page='+ str(i+1), headers=headers)
    html = response.content

    # Decodificando o HTML:
    html = html.decode('utf-8')

    # Tratando o HTML:
    html = trata_html(html)

    #
    soup = BeautifulSoup(html, 'html.parser')


    # Obtendo as informações de cada um dos restaurantes:
    dados_restaurantes = soup.find('div', {'class': 'ui cards'}).findAll('div', class_='col-s-16 col-m-12 pl0')
    tipos_restaurantes = soup.find('div', {'class': 'ui cards'}).findAll('div', class_='search-page-text clearfix row')


    # Coletando as informações de cada restaurante:
    n_restaurantes_pagina = len(dados_restaurantes)

    for restaurante in range(n_restaurantes_pagina):
        card = {}
        
        # Informações sobre os restaurantes:
        card['restaurante'] = dados_restaurantes[restaurante].find('a', {'class':'result-title hover_feedback zred bold ln24 fontsize0'}).getText()
        if dados_restaurantes[restaurante].find('span', {'class':'rating-value'}) is not None:
            card['nota'] = dados_restaurantes[restaurante].find('span', {'class':'rating-value'}).getText()
        else:
            'None'
        if dados_restaurantes[restaurante].find('span', {'class':'review-count medium'}) is not None:
            card['numero_de_avaliacoes'] = dados_restaurantes[restaurante].find('span', {'class':'review-count medium'}).getText()
        else:
            'None'
        if dados_restaurantes[restaurante].find('a', {'class':'zdark ttupper fontsize6'}) is not None:
            card['tipo'] = dados_restaurantes[restaurante].find('a', {'class':'zdark ttupper fontsize6'}).getText()
        else:
            'None'
        if tipos_restaurantes[restaurante].find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}) is not None:
            card['cozinha'] = tipos_restaurantes[restaurante].find('span', {'class':'col-s-11 col-m-12 nowrap pl0'}).getText()
        else:
            'None'
        if tipos_restaurantes[restaurante].find('span', {'class':'col-s-11 col-m-12 pl0'}) is not None:
            card['custo_para_dois'] = tipos_restaurantes[restaurante].find('span', {'class':'col-s-11 col-m-12 pl0'}).getText()
        else:
            'None'
        if dados_restaurantes[restaurante].find('a', {'class':'ln24 search-page-text mr10 zblack search_result_subzone left'}) is not None:
            card['bairro'] = dados_restaurantes[restaurante].find('a', {'class':'ln24 search-page-text mr10 zblack search_result_subzone left'}).getText()
        else:
            'None'

        # Adicionando resultado a lista CARD
        cards.append(card)
        
# Criando um DataFrame com os resultados
dataset_restaurantes = pd.DataFrame(cards)
dataset_restaurantes

,restaurante,nota,numero_de_avaliacoes,tipo,cozinha,custo_para_dois,bairro
0,A Figueira Rubaiyat,4.4,(51 Opiniões),Refeições de luxo,"Churrasco, Grelhados, Brasileira",R$ 300,Jardim Paulista
1,Terraço Itália,4.2,(42 Opiniões),Refeições de luxo,Italiana,R$ 400,República
2,Veloso,4.1,(64 Opiniões),Bar,"Brasileira, Comida de bar, Bebidas",R$ 70,Vila Mariana
3,Skye - Hotel Unique,4.0,(63 Opiniões),Lounge,"Bebidas, Internacional",R$ 300,"Hotel Unique, Jardim Paulista"
4,Coco Bambú,4.0,(41 Opiniões),Refeições casuais,Marisqueira,R$ 200,Itaim Bibi
...,...,...,...,...,...,...,...
2245,Parrilla São Paulo,NaN,NaN,Refeições casuais,Argentina,R$ 130,Pinheiros
2246,Ambrosia,NaN,NaN,Refeições casuais,Brasileira,R$ 75,Itaim Bibi
2247,Bella Praça,NaN,NaN,Refeições rápidas,"Internacional, Brasileira",R$ 60,"Bela Vista, Centro"
2248,Grão Espresso,NaN,NaN,Café,Pastelaria,R$ 35,Itaim Bibi


In [84]:
dataset_restaurantes.to_csv('dataset_restaurantes_sujo.csv')

### Tratamento da base:

In [85]:
dataset_restaurantes = pd.read_csv("dataset_restaurantes_sujo.csv")
dataset_restaurantes.head()

,Unnamed: 0,restaurante,nota,numero_de_avaliacoes,tipo,cozinha,custo_para_dois,bairro
0,0,A Figueira Rubaiyat,4.4,(51 Opiniões),Refeições de luxo,"Churrasco, Grelhados, Brasileira",R$ 300,Jardim Paulista
1,1,Terraço Itália,4.2,(42 Opiniões),Refeições de luxo,Italiana,R$ 400,República
2,2,Veloso,4.1,(64 Opiniões),Bar,"Brasileira, Comida de bar, Bebidas",R$ 70,Vila Mariana
3,3,Skye - Hotel Unique,4.0,(63 Opiniões),Lounge,"Bebidas, Internacional",R$ 300,"Hotel Unique, Jardim Paulista"
4,4,Coco Bambú,4.0,(41 Opiniões),Refeições casuais,Marisqueira,R$ 200,Itaim Bibi


In [86]:
# Retirando a coluna 'Unnamed: 0':
dataset_restaurantes.drop(columns="Unnamed: 0", inplace=True)
dataset_restaurantes.head()

,restaurante,nota,numero_de_avaliacoes,tipo,cozinha,custo_para_dois,bairro
0,A Figueira Rubaiyat,4.4,(51 Opiniões),Refeições de luxo,"Churrasco, Grelhados, Brasileira",R$ 300,Jardim Paulista
1,Terraço Itália,4.2,(42 Opiniões),Refeições de luxo,Italiana,R$ 400,República
2,Veloso,4.1,(64 Opiniões),Bar,"Brasileira, Comida de bar, Bebidas",R$ 70,Vila Mariana
3,Skye - Hotel Unique,4.0,(63 Opiniões),Lounge,"Bebidas, Internacional",R$ 300,"Hotel Unique, Jardim Paulista"
4,Coco Bambú,4.0,(41 Opiniões),Refeições casuais,Marisqueira,R$ 200,Itaim Bibi


In [87]:
# Deixando apenas a quantidade de avaliações na variável 'numero_de_avaliacoes':
if dataset_restaurantes['numero_de_avaliacoes'].empty:
    dataset_restaurantes['numero_de_avaliacoes'] = "NaN"
else:
    dataset_restaurantes['numero_de_avaliacoes'] = dataset_restaurantes['numero_de_avaliacoes'].str.split(pat=" ", expand=True)
    
if dataset_restaurantes['numero_de_avaliacoes'].empty:
    dataset_restaurantes['numero_de_avaliacoes'] = "NaN"
else:
    dataset_restaurantes['numero_de_avaliacoes'] = dataset_restaurantes['numero_de_avaliacoes'].str[1:]
dataset_restaurantes.head()

,restaurante,nota,numero_de_avaliacoes,tipo,cozinha,custo_para_dois,bairro
0,A Figueira Rubaiyat,4.4,51,Refeições de luxo,"Churrasco, Grelhados, Brasileira",R$ 300,Jardim Paulista
1,Terraço Itália,4.2,42,Refeições de luxo,Italiana,R$ 400,República
2,Veloso,4.1,64,Bar,"Brasileira, Comida de bar, Bebidas",R$ 70,Vila Mariana
3,Skye - Hotel Unique,4.0,63,Lounge,"Bebidas, Internacional",R$ 300,"Hotel Unique, Jardim Paulista"
4,Coco Bambú,4.0,41,Refeições casuais,Marisqueira,R$ 200,Itaim Bibi


In [88]:
# Simplificando a variável que caracteriza o tipo de restaurante:
tipos = {
    'Refeições de luxo' : 'Luxo',
    'Refeições casuais' : 'Casual',
    'Refeições rápidas' : 'Rápido',
    'Bar de cocktails' : 'Bar'
}

dataset_restaurantes['tipo'] = dataset_restaurantes['tipo'].replace(tipos)

In [89]:
dataset_restaurantes.head()

,restaurante,nota,numero_de_avaliacoes,tipo,cozinha,custo_para_dois,bairro
0,A Figueira Rubaiyat,4.4,51,Luxo,"Churrasco, Grelhados, Brasileira",R$ 300,Jardim Paulista
1,Terraço Itália,4.2,42,Luxo,Italiana,R$ 400,República
2,Veloso,4.1,64,Bar,"Brasileira, Comida de bar, Bebidas",R$ 70,Vila Mariana
3,Skye - Hotel Unique,4.0,63,Lounge,"Bebidas, Internacional",R$ 300,"Hotel Unique, Jardim Paulista"
4,Coco Bambú,4.0,41,Casual,Marisqueira,R$ 200,Itaim Bibi


In [90]:
# Transformando a variável 'tipo' em uma variável dummy:
dummies = pd.get_dummies(dataset_restaurantes['tipo'])
dataset_restaurantes = pd.concat([dataset_restaurantes, dummies], axis=1)

In [91]:
dataset_restaurantes.drop(columns='tipo', inplace=True)

In [92]:
nome_colunas_dummies = {
    'Bar': 'tipo_bar',
    'Café': 'tipo_cafe',
    'Casual': 'tipo_casual',
    'Club': 'tipo_club',
    'Lounge': 'tipo_lounge',
    'Luxo': 'tipo_luxo',
    'Padaria': 'tipo_padaria',
    'Pub': 'tipo_pub',
    'Rápido': 'tipo_rapido',
    'Sobremesas': 'tipo_sobremesas',
    'Praça de alimentação': 'tipo_praca_alimentacao'
}

dataset_restaurantes.rename(mapper=nome_colunas_dummies, axis=1, inplace=True)

In [93]:
# Retirando o R$ da variável 'custo_para_dois':
if dataset_restaurantes['custo_para_dois'].empty:
    dataset_restaurantes['custo_para_dois'] = "NaN"
else:
    dataset_restaurantes['custo_para_dois'] = dataset_restaurantes['custo_para_dois'].str[3:]
dataset_restaurantes.head()

,restaurante,nota,numero_de_avaliacoes,cozinha,custo_para_dois,bairro,tipo_bar,tipo_cafe,tipo_casual,tipo_club,tipo_lounge,tipo_luxo,tipo_padaria,tipo_praca_alimentacao,tipo_pub,tipo_rapido,tipo_sobremesas,Wine Bar
0,A Figueira Rubaiyat,4.4,51,"Churrasco, Grelhados, Brasileira",300,Jardim Paulista,0,0,0,0,0,1,0,0,0,0,0,0
1,Terraço Itália,4.2,42,Italiana,400,República,0,0,0,0,0,1,0,0,0,0,0,0
2,Veloso,4.1,64,"Brasileira, Comida de bar, Bebidas",70,Vila Mariana,1,0,0,0,0,0,0,0,0,0,0,0
3,Skye - Hotel Unique,4.0,63,"Bebidas, Internacional",300,"Hotel Unique, Jardim Paulista",0,0,0,0,1,0,0,0,0,0,0,0
4,Coco Bambú,4.0,41,Marisqueira,200,Itaim Bibi,0,0,1,0,0,0,0,0,0,0,0,0


In [94]:
# Transformando a variável 'cozinha' em uma variável dummy:
dataset_restaurantes['cozinha'] = ', ' + dataset_restaurantes['cozinha']
# (Inserindo uma vírgula antes dos caracteres da coluna para dividi-las em dummies posteriormente.)

dummies_cozinha = dataset_restaurantes['cozinha'].str.get_dummies(sep=",")

dummies_cozinha.columns = dummies_cozinha.columns.str.lower()

nomes_colunas_dummies_cozinha = {
    ' alemã': 'coz_alema',
    ' americana': 'coz_americana',
    ' argentina': 'coz_argentina',
    ' asiática': 'coz_asiatica',
    ' australiana': 'coz_australiana',
    ' autor': 'coz_autor',
    ' baiana': 'coz_baiana',
    ' bebidas': 'coz_bebidas',
    ' bifes': 'coz_bifes',
    ' brasileira': 'coz_brasileira',
    ' brasserie': 'coz_brasserie',
    ' cafeteria': 'coz_cafeteria',
    ' caipira': 'coz_caipira',
    ' chinesa': 'coz_chinesa',
    ' churrasco': 'coz_churrasco',
    ' comida de bar': 'coz_comida_de_bar',
    ' comida saudável': 'coz_comida_saudavel',
    ' contemporânea': 'coz_contemporanea',
    ' coreana': 'coz_coreana',
    ' delicatessen': 'coz_delicatessen',
    ' egípcia': 'coz_egipcia',
    ' espanhola': 'coz_espanhola',
    ' fast food': 'coz_fast_food',
    ' fondue': 'coz_fondue',
    ' francesa': 'coz_francesa',
    ' fusão': 'coz_fusao',
    ' gaúcha': 'coz_gaucha',
    ' gelados': 'coz_gelados',
    ' gourmet fast food': 'coz_gourmet_fast_food',
    ' grelhados': 'coz_grelhados',
    ' hamburgueria': 'coz_hamburgueria',
    ' húngara': 'coz_hungara',
    ' inglesa': 'coz_inglesa',
    ' internacional': 'coz_internacional',
    ' italiana': 'coz_italiana',
    ' japonesa': 'coz_japonesa',
    ' kebab': 'coz_kebab',
    ' libanesa': 'coz_libanesa',
    ' marisqueira': 'coz_marisqueira',
    ' marroquina': 'coz_marroquina',
    ' mediterrânica': 'coz_mediterranica',
    ' mexicana': 'coz_mexicana',
    ' mineira': 'coz_mineira',
    ' nordestina': 'coz_nordestina',
    ' padaria': 'coz_padaria',
    ' pastelaria': 'coz_pastelaria',
    ' patisserie': 'coz_patisserie',
    ' peixe fresco': 'coz_peixe_fresco',
    ' peruana': 'coz_peruana',
    ' pizza': 'coz_pizza',
    ' portuguesa': 'coz_portuguesa',
    ' sandwich': 'coz_sandwich',
    ' sobremesa': 'coz_sobremesa',
    ' street food': 'coz_street_food',
    ' sul americana': 'coz_sul_americana',
    ' sumos': 'coz_sumos',
    ' sushi': 'coz_sushi',
    ' tailandesa': 'coz_tailandesa',
    ' tapas': 'coz_tapas',
    ' vegetariana': 'coz_vegetariana',
    ' árabe': 'coz_arabe',
    ' africana': 'coz_africana',
    ' capixaba': 'coz_capixaba',
    ' chilena': 'coz_chilena',
    ' chás': 'coz_chas',
    ' europeia': 'coz_europeia',
    ' grega': 'coz_grega',
    ' indiana': 'coz_indiana',
    ' irlandesa': 'coz_irlandesa',
    ' mongol': 'coz_mongol',
    ' russa': 'coz_russa',
    ' suiça': 'coz_suica',
    ' só bebidas': 'coz_so_bebidas',
    ' turca': 'coz_turca', 
    ' arménia': 'coz_armenia'
}

dummies_cozinha.rename(mapper=nomes_colunas_dummies_cozinha, axis=1, inplace=True)

dataset_restaurantes = pd.concat([dataset_restaurantes, dummies_cozinha], axis=1)

dataset_restaurantes.drop(columns='cozinha', inplace=True)

dataset_restaurantes.head()

,restaurante,nota,numero_de_avaliacoes,custo_para_dois,bairro,tipo_bar,tipo_cafe,tipo_casual,tipo_club,tipo_lounge,...,coz_suica,coz_sul_americana,coz_sumos,coz_sushi,coz_so_bebidas,coz_tailandesa,coz_tapas,coz_turca,coz_vegetariana,coz_arabe
0,A Figueira Rubaiyat,4.4,51,300,Jardim Paulista,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Terraço Itália,4.2,42,400,República,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Veloso,4.1,64,70,Vila Mariana,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Skye - Hotel Unique,4.0,63,300,"Hotel Unique, Jardim Paulista",0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Coco Bambú,4.0,41,200,Itaim Bibi,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
dataset_restaurantes.to_csv('dataset_restaurantes.csv')